In [6]:
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict
class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]

In [ ]:
from typing import TypedDict, Annotated

# Function to define the custom append behavior
def add_messages(existing: list, new: list) -> list:
    # Appends new messages to the existing list
    return existing + new

# Define the TypedDict
class State(TypedDict):
    # Annotate 'messages' with the custom update function
    messages: Annotated[list, add_messages]

# Initialize the state with an existing list of messages
state: State = {"messages": ["Hello"]}

# Define the new data to "push" (append to the list)
new_messages = ["How are you?", "Good to see you!"]

# Use the `add_messages` function to update the `messages` key
state["messages"] = add_messages(state["messages"], new_messages)

# Print the updated state
print(state)

In [17]:
from typing import Annotated
from pydantic import BaseModel, Field

class User(BaseModel):
    # Annotated with Field for constraints
    age: Annotated[int, Field(ge=18, le=100)]  # Age must be between 18 and 100
    name: Annotated[str, Field(min_length=3, max_length=50)]  # Name length must be 3-50

# Example usage
user = User(age=25, name="Rakesh")
print(user)

# Validation error example
# invalid_user = User(age=15, name="A")  # Raises ValidationError

age=25 name='Rakesh'


In [19]:
# Validation error example
# invalid_user = User(age=15, name="A")  # Raises ValidationError

In [34]:
from pydantic import BaseModel, Field, field_validator

class Item(BaseModel):
    price: float = Field(gt=0)  # Basic validation: price > 0

    # New-style validator
    @field_validator("price")
    def must_be_positive(cls, value):
        if value <= 0:
            raise ValueError("Price must be positive!")
        return value

# Valid Item
item = Item(price=10)
print(item)

# Invalid Item
try:
    invalid_item = Item(price=-5)
except Exception as e:
    print("Error:", e)

price=10.0
Error: 1 validation error for Item
price
  Input should be greater than 0 [type=greater_than, input_value=-5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/greater_than


In [25]:
# invalid_item = Item(price=-10)  # Raises ValidationError
# print(invalid_item)

price=-10


In [30]:
from typing import Annotated
from pydantic import BaseModel, Field, model_validator

class Product(BaseModel):
    name: Annotated[str, Field(min_length=3, max_length=50)]
    price: Annotated[float, Field(gt=0)]  # Price must be greater than 0
    discount: Annotated[float, Field(ge=0, le=1)]  # Discount must be between 0 and 1

    @model_validator(mode='Any')
    def validate_discounted_price(cls, values):
        price = values.get("price")
        discount = values.get("discount")
        if price and discount and price * (1 - discount) < 10:
            raise ValueError("Discounted price must be at least 10.")
        return values

# Example usage
product = Product(name="Laptop", price=1000, discount=0.1)  # Valid
# invalid_product = Product(name="Laptop", price=50, discount=0.9)  # ValidationError

AssertionError: invalid mode: 'Any', expected 'before', 'after' or 'plain